<a href="https://colab.research.google.com/github/ChKik/thesis_project/blob/development/MobileNetV2_tflite_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input #gia to -1,1 scaling twn pixel poy den eixa valei pio prin 22/7
from tensorflow.keras import layers, models, regularizers, Input #l2 regularization gia to overfitting 22/7
# gia tis katigories, tis kanei one hot encoding.
# tha einai se ena binary numpy array me plithos column oses oi katigories
import numpy as np
import platform
import pathlib
import pandas as df
from PIL import Image
import os
import random

#checkarw ta versions poy xrisimopoiei to collab gia na apofugw conflicts.
print('Python version:', platform.python_version())
print("TF:", tf.__version__)  #  version 2.18.0
print("NumPy:", np.__version__)  #  version 2.0.2
print('Keras version:', tf.keras.__version__)

In [ ]:
from google.colab import drive #only do it once at the start.
drive.mount('/content/drive')

In [ ]:
drive_path= "drive/My Drive/Diplomatiki/hand_gesture_dataset_resized.zip"

!unzip -q "{drive_path}" -d . #to kanei unzip kai to vazei sto current directory to file . Epeidh evala . ginetai ayto.


In [ ]:
def augment_image(image): #kanei random flip,brightness ktlp tis fotografies kai tis allazei gia na kanoyme prevent to overfit. 22/7
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, 0.9, 1.1)
    image = tf.image.random_saturation(image, 0.9, 1.1)
    return image

# Dataset path sto google drive
base_path = "/content/hand_gesture_dataset_resized"

#TRAINING FOR MobileNetV2 model which needs [-1,1] scaling for pixels (not 0-1 if using pretrained weights)
def image_generator(subfolder, batch_size=32):
    """Generator that yields batches of images and labels on-demand"""
    class_folders = [str(i) for i in range(20)]  # All class folders (0-19)

    while True:  # Infinite loop for Keras compatibility
        print("New Epoch")
        for class_label, class_name in enumerate(class_folders):
            class_path = os.path.join(base_path, subfolder, class_name)

            if not os.path.exists(class_path):
                continue

            batch_images = []
            batch_labels = []
            print("Class label " + str(class_label)) # Convert class_label to string, thelei kai typecasting.
            for img_name in os.listdir(class_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_path, img_name)
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224)) #exei gisei hdh apo to script alla making sure

                    img_array = tf.keras.preprocessing.image.img_to_array(img)  # load as array (range [0,255])
                    img_array = tf.cast(img_array, tf.float32) #dokimazw na to kanw se float iws.
                    img_array = augment_image(img_array) #Augment the jpg before preprocessing to try to prevent overfitting.Tha to testarw mono me ayto.
                    img_array = preprocess_input(img_array)  #  scale to [-1,1] gia MobileNetV2
                    batch_images.append(img_array)
                    batch_labels.append(class_label)

                    #stop otan exoyme arketa eimages, edw 32
                    if len(batch_images) == batch_size:
                        yield (np.array(batch_images),
                               to_categorical(batch_labels, num_classes=20)) #onehot enc gia tis katigories kai einai np array. binary array me columns=classes num
                        batch_images = []
                        batch_labels = []

            # Yield remaining images in partial batch
            if batch_images:
                yield (np.array(batch_images),
                       to_categorical(batch_labels, num_classes=20))

# Create generators instead of loading all data at once,fernei mexri 32 images kathe fora kai ekpaidevetai stadiaka.
train_gen = image_generator("train", batch_size=32)
test_gen = image_generator("test", batch_size=32)

# Count total images (optional - for progress tracking)
def count_images(subfolder):
    count = 0
    for class_label in range(20):
        class_path = os.path.join(base_path, subfolder, str(class_label))
        if os.path.exists(class_path):
            count += len([f for f in os.listdir(class_path)
                         if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    return count

train_count = count_images("train")
test_count = count_images("test")
print(f"Total training images: {train_count}")
print(f"Total test images: {test_count}")

# Create tf.data.Dataset from generators
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=((None, 224, 224, 3), (None, 20))
).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: test_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=((None, 224, 224, 3), (None, 20))
).prefetch(tf.data.AUTOTUNE)

print("\nDatasets ready for training (RAM efficient)")


Total training images: 18000
Total test images: 6000

Datasets ready for training (RAM efficient)


In [ ]:
model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),  # Must specify when include_top=True
    alpha=0.5,  #back sto 0.35 to alpha, dokimazw to 0.5 23-7
    include_top=False,  #Remove the original 1000-class layer
    weights="imagenet")

#Custom head gia 20 classes mono
# Input layer
inputs = tf.keras.Input(shape=(224, 224, 3))  #layer0

# Pass images through the base model
x = model(inputs, training=False)  # Freeze to layer 1 gia na min xasw ta varh, layer1

# Add new layers:
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Reduces spatial dimensions apo 3d tis kanei 1d  , layer 2
x = tf.keras.layers.Dropout(0.3)(x)              # Regularization to prevent overfitting, kanei merika layers drop, layer 3. Auksanw to dropout se 0.4->0.3 22/7
outputs = tf.keras.layers.Dense(20, activation='softmax'      #,kernel_regularizer=regularizers.l2(0.01)  remove to l2 reg for now.
)(x)  # 20-class prediction  , layer 4 , l2 regularization


#Dokimzw kai early stop poy kanei monitor to val loss
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)
model = tf.keras.Model(inputs, outputs, name="Zephyr_MobileNetV2")


In [ ]:
#Dokimzw kai early stop poy kanei monitor to val loss
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

#epeidh gia kapoio logo vazei CPU by default checkarei ama exeis valei gpu gia to runtime
print("GPU Available:", tf.config.list_physical_devices('GPU'))


model.layers[1].trainable = False  # Freeze MobileNetV2 (layer 1 stin periptwsh mas gia na min xalasei sto new training)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#gia ta vimata twn epoch
steps_per_epoch = train_count // 32
validation_steps = test_count // 32

#train to modelo edw.

# 1. Freeze base and train head
print("Training starts")
model.layers[1].trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit( train_dataset,
    steps_per_epoch=steps_per_epoch,
    epochs=20,  #dokimazw na kanw increase ta epoch apo 10 se 20 mpas kai exw kalutera results sto training
    validation_data=test_dataset,
    validation_steps=validation_steps,
    callbacks=[
        tf.keras.callbacks.TerminateOnNaN(),
        tf.keras.callbacks.ModelCheckpoint('head_trained.keras') #to allaksa apo h5 se keras gia next , alla exw h5 twra.
    ])  #evala kai vimata twra. Episis krataei ta kalutera weights se periptosi poy crasharei h to accuracy einai NaN

# 2. Unfreeze kai to prwto layer and fine-tune to modelo
print("Fine tuning starts")
model.layers[1].trainable = True
model.compile(  optimizer=Adam(1e-5),  # Much lower learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy'])  # poly mikrotero learning apo prin

print("Fine tuning begins")
model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    callbacks=[
        tf.keras.callbacks.TerminateOnNaN(),
        tf.keras.callbacks.ModelCheckpoint('fine_tuned.keras'), #to allaksa apo h5 se keras gia next , alla exw h5 twra.
        early_stop
    ])

print("Training done ")

In [ ]:
#Quantization process now to convert all the float values into uint8 to reduce the size of inference.
# ======================
#  Post-training Quantization
# Gia to teliko apotelesma tha exoyme inputs/outputs se range 0-255 uint8 gia na meiwsoyme to size anti gia float. Meiwsh apo 5.2mb se 658 kb etsi.
# ======================

def representative_dataset():
    for images, _ in train_dataset.take(100):  # About 100 batches * 32 = ~3200 images
        for i in range(images.shape[0]):
            yield [images[i:i+1]]

# Create TFLite converter from the trained Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable optimizations (this activates quantization)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Provide representative dataset for activation quantization
converter.representative_dataset = representative_dataset

# Set inference input/output types to uint8 for full integer quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8  # tha exei san input kai output to model uint8
converter.inference_output_type = tf.uint8

# Convert the model
quantized_tflite_model = converter.convert()


with open("zephyr_quantized_int8.tflite", "wb") as f:
    f.write(quantized_tflite_model)

print("Quantized model saved as zephyr_quantized_int8.tflite")


In [ ]:
#This cell ensures that the model expects the input image of 0-255 quantized uint8 image I am gonna pass to it.
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/Diplomatiki/output/zephyr_quantized_int8.tflite")
interpreter.allocate_tensors()
input_info = interpreter.get_input_details()[0]
print(input_info['dtype'], input_info['quantization'])